# 1.Definição da Classe (Algoritmo) que seleciona a Base de Dados para Download. 

As bases disponíveis são:

1.   GeoSES
2.   SIM
3.   ETLCNES
4.   PolisPCDaS
5.   ETLSIH 

A fonte dos dados é: https://pcdas.icict.fiocruz.br/conjunto-de-dados/



In [ ]:
import pandas as pd
import zipfile
import time
!pip install wget --quiet
import wget
import numpy as np
import os
from tqdm import tqdm
import json
import urllib
import os
import json
!pip install --upgrade xlrd

class OOBr:
  def __init__(self,uf = "" ):
    '''
    Inicializa a classe, solicitando a escolha de qual base deseja-se trabalhar
    '''
    os.system("rm -Rf sample_data")
    self.download_indicadores()
    pass
    

  
  def bar_custom(self, current, total, width=80):
    '''
      Função da Barra de status do download
    '''
    print("Downloading: %d%% [%d / %d] bytes" % (current / total * 100,
                                                current, total))
      


  def descompactar(self, nome_arquivo):
    '''
    Função Descompactadora de arquivos zips
    '''
    os.system("mkdir workdir")
    with zipfile.ZipFile(nome_arquivo, 'r') as zip_ref:
      zip_ref.extractall("./workdir/")
  #descompactar('./vetores_516_enem_2019.zip')

  
  def descompacta_download(self, url):
    '''
    Função de Interface pra Descompactação do Download
    '''
    start_time = time.time()  
    self.descompactar(url.split("/")[-1])
    print("Tempo de descompactação %s segundos de descompactação, isto é: %s minutos"\
            % (round(time.time() - start_time,2), round((time.time() - start_time)/60)))
    os.system("rm "+url.split("/")[-1])


 
  def download_indicadores(self, ):
    '''
      Função de download dos indicadores a partida url das bases:
      bases = { 'GEOSES': 'https://sage.saude.gov.br/sistemas/geoses/scores.zip',
              'SIM' : 'https://bigdata-arquivos.icict.fiocruz.br/SIM/ETLSIM.zip',
              'SINASC': 'https://bigdata-arquivos.icict.fiocruz.br/SINASC/ETLSINASC.zip', 
              'ETLCNES' : 'https://bigdata-arquivos.icict.fiocruz.br/CNES/ETLCNES.zip', 
              'PolisPCDaS': 'https://bigdata-arquivos.icict.fiocruz.br/PolisPCDaS/PolisPCDaS.zip' 
            #  'ETLSIH': 'https://bigdata-arquivos.icict.fiocruz.br/SIH/ETLSIH.zip'
            }
    '''
    bases = { 'GEOSES': 'https://sage.saude.gov.br/sistemas/geoses/scores.zip',
              'SIM' : 'https://bigdata-arquivos.icict.fiocruz.br/SIM/ETLSIM.zip',
              'SINASC': 'https://bigdata-arquivos.icict.fiocruz.br/SINASC/ETLSINASC.zip', 
              'ETLCNES' : 'https://bigdata-arquivos.icict.fiocruz.br/CNES/ETLCNES.zip', 
              'PolisPCDaS': 'https://bigdata-arquivos.icict.fiocruz.br/PolisPCDaS/PolisPCDaS.zip' 
            #  'ETLSIH': 'https://bigdata-arquivos.icict.fiocruz.br/SIH/ETLSIH.zip'
            }
    print('Informe o número da base que deseja acessar')
    bases_nomes = list(bases.keys())
    for i in enumerate(bases_nomes):
      print("%d %s" % (i[0], i[1]))
    print("-1 Reset Data")

    op = int(input(""))
    if op ==-1:
      self.clean_dados()
    else:
      try:
        url = bases[bases_nomes[op]]
        print("Download da base %s" % url.split("/")[-1])
        start_time = time.time()    
        wget.download(url, bar=self.bar_custom)  
        #wget.download(url)  
        print("Descompactando da base %s" % url.split("/")[-1])
        self.descompacta_download(url)
        print("Tempo de download %s segundos de descompactação, isto é: %s minutos"\
              % (round(time.time() - start_time,2), round((time.time() - start_time)/60)))
        
        print("Os dados baixados encontram-se no diretório workdir")
        print(bases_nomes[op])
        self.inspeciona_dados(bases_nomes[op]) 
        self.download_ibge_nome_municipios()
      except:
        print("Não foi possível realizar a operação")


 
  def inspeciona_dados(self, base_nome):
    '''
    Função que retorna lista de tabelas baixadas
    '''
    bases = { 'GEOSES': './workdir/scores/',
              'SIM' : './workdir/',
              'SINASC': './workdir/',
              'ETLCNES' : './workdir/home/dataiku/dss/exports/data', 
              'PolisPCDaS': './workdir/', 
              'ETLSIH': './workdir/',
            }
    

    
    #all_files = os.listdir("./workdir/")    
    all_files = os.listdir(bases[base_nome])    
    planilhas_sim = {}
    csv_files = list(filter(lambda f: f.endswith('.csv'), all_files))
    csv_files.sort()


    self.lista_planilhas =  csv_files.copy()
    print("Primeiras 3 ", self.lista_planilhas[:3])
    print("Últimas 3 ",self.lista_planilhas[-3:])


  
  def sim_load(self, uf=[], ano=[], proporcao=30):
    '''
      Função que carrega em memória os dados da base SIM, no formato de dataframe
    ''' 
    aux = []  
    if len(uf) == 0 and len(ano)==0:    
      print('Escolha ao menos um estado ou ano')
    elif len(uf) > 0 and len(ano)==0:    
      for i in  self.lista_planilhas:
        for j in uf:
          if j in i.replace("ETLSIM.DORES_", "").replace("_t.csv", ""):
            aux.append(i)
    elif len(uf) == 0 and len(ano)>0:    
      for i in  self.lista_planilhas:
        for j in ano:
          if str(j) in i.replace("ETLSIM.DORES_", "").replace("_t.csv", ""):
            aux.append(i)
    else:    
      for i in  self.lista_planilhas:
        for j in uf:
          for m in ano:
            if j in i.replace("ETLSIM.DORES_", "").replace("_t.csv", "") and str(m) in i.replace("ETLSIM.DORES_", "").replace("_t.csv", ""):
              aux.append(i)
                

    

    raw_data = {}
    
    print(aux)
    for i in aux:         
      _ = pd.read_csv("./workdir/"+i, sep=',')
      #_ = pd.read_csv("./workdir/"+i, nrows=100, sep=',')
      linhas = _.shape[0]
      #raw_data[i.replace("ETLSIM.DORES_", "").replace("_t.csv", "")] = _
      print(i)
      raw_data[i.replace("ETLSIM.DORES_", "").replace("_t.csv", "")] = _.sample(int(linhas * proporcao/100))
    self.sim_planilhas = raw_data
    self.dict_SIM = pd.read_csv("./workdir/dict_SIM.csv", sep=',')
    

    
  def sinasc_load(self, uf=[], ano=[], proporcao=30):
    '''
      Função que carrega em memória os dados da base SINASC, no formato de dataframe
    '''
    aux = []  
    if len(uf) == 0 and len(ano)==0:    
      print('Escolha ao menos um estado ou ano')
    elif len(uf) > 0 and len(ano)==0:    
      for i in  self.lista_planilhas:
        for j in uf:
          if j in i.replace("ETLSINASC.DNRES_", "").replace("_t.csv", ""):
            aux.append(i)
    elif len(uf) == 0 and len(ano)>0:    
      for i in  self.lista_planilhas:
        for j in ano:
          if str(j) in i.replace("ETLSINASC.DNRES_", "").replace("_t.csv", ""):
            aux.append(i)
    else:    
      for i in  self.lista_planilhas:
        for j in uf:
          for m in ano:
            if j in i.replace("ETLSINASC.DNRES_", "").replace("_t.csv", "") and str(m) in i.replace("ETLSIM.DORES_", "").replace("_t.csv", ""):
              aux.append(i)
                
  
    

    raw_data = {}
    
    #print(aux)
    for i in aux:         
      _ = pd.read_csv("./workdir/"+i, sep=',')
      #_ = pd.read_csv("./workdir/"+i, nrows=100, sep=',')
      linhas = _.shape[0]
      #raw_data[i.replace("ETLSIM.DORES_", "").replace("_t.csv", "")] = _
      #print(i)
      raw_data[i.replace("ETLSINASC.DNRES_", "").replace("_t.csv", "")] = _.sample(int(linhas * proporcao/100))
    self.sinasc_planilhas = raw_data
    #self.dict_SIM = pd.read_csv("./workdir/dict_SIM.csv", sep=',')



  def download_ibge_nome_municipios(self,):
    os.system("wget https://geoftp.ibge.gov.br/organizacao_do_territorio/estrutura_territorial/divisao_territorial/2021/DTB_2021.zip")
    os.system("unzip DTB_2021.zip -d ./workdir")
    os.system("rm DTB_2021.zip")

    self.df_cod_mun_ibge = pd.read_excel("./workdir/RELATORIO_DTB_BRASIL_MUNICIPIO.xls")
    self.df_cod_mun_ibge['ibge_'] = self.df_cod_mun_ibge['Código Município Completo'].astype(str)
    novo = []
    for i in self.df_cod_mun_ibge['ibge_'].values:
      novo.append(i[:-1])
    self.df_cod_mun_ibge['cod_6'] = novo
    self.df_cod_mun_ibge['cod_6'] = self.df_cod_mun_ibge['cod_6'].astype(int)

  def load_geoses(self, uf=[]):
    '''
    Função que carrega em memória os dados da base GeoSES, no formato de dataframe
    '''
    aux = []  
    if len(uf) == 0:    
      print('Escolha ao menos um estado ou ano')
    elif len(uf) > 0:    
      for i in  self.lista_planilhas[-28:]:
        for j in uf:
          if j in i:
            aux.append(i)

    raw_data = {}
      
    #print(aux)
    for i in aux:         
      _ = pd.read_csv("./workdir/scores/"+i, sep=',')
      #_ = pd.read_csv("./workdir/"+i, nrows=100, sep=',')
      
      #raw_data[i.replace("ETLSIM.DORES_", "").replace("_t.csv", "")] = _
      #print(i)
      raw_data[i.replace(".csv", "")] = _
    self.geoses_planilhas = raw_data

  

  def get_nome_municipio(self, id):
    
    url = "https://servicodados.ibge.gov.br/api/v1/localidades/municipios/"+str(id)

    result = os.popen("curl " + url).read()
    
    json_acceptable_string = result.replace("'", "\"")
    #print(json_acceptable_string)
    try:
      d = json.loads(json_acceptable_string)
      return d['nome']
    except:
      return str(id)
  




  def clean_dados(self, ):
    '''
        Função que limpa os dados coletados da PCDaS
    '''
    os.system("rm -Rf workdir")
    print("workdir removido")



## 1.1) Informações sobre a classe OOBr e seus métodos

In [ ]:
help(OOBr)

## 1.2) Método Principal - Escolha da Base a ser Trabalhada

In [ ]:
oobr = OOBr()

## 1.3) Reset do ambiente

Remove todos os arquivos dentro do diretório workidr (que é onde são depositados os arquivos após download e descompcatação)

In [ ]:
oobr.clean_dados()

# 2) Base de Dados GeoSES

Selecione a opção 0 para download da base GeoSES



In [ ]:
oobr = OOBr()

## 2.1) Listagem das Planilhas Geoses - Estado e Nacional

O comando a seguir mostra a lista de planilhas contidas na base GeoSES. No caso, estão sendo listadas as 30 últimas, da lista ordenada alfabeticamente. 

In [ ]:
print("Quantidade de planilhas na base GeoSES %d" % len(oobr.lista_planilhas))
print("Listagem das 30 últimas planilhas da lista ordenada:")
oobr.lista_planilhas[-30:]

## 2.2) Criação da lista com siglas dos estados com indicadores GeoSES

In [ ]:
estados = [s.replace(".csv", "") for s in oobr.lista_planilhas[-28:]]
estados

## 2.3) Carga dos Dados GeoSES por Estado

In [ ]:
'''
Carga dos dados geoses
'''
oobr.load_geoses(uf=estados)



## 2.3.a) Listagem dos Planilhas Carregadas

In [ ]:
oobr.geoses_planilhas.keys()

## 2.3.b) Listagem de uma Planilha Carregada

In [ ]:
oobr.geoses_planilhas['AC']

## 2.3.c) Exportar uma Planilha Carregada

In [ ]:
from google.colab import files
oobr.geoses_planilhas['AC'].to_csv("geoses.csv", sep=';', index=False)

In [ ]:
files.download('geoses.csv')

## 2.4) Inserção do Nome do Município

Utilizando a tabela de municípios do IBGE, fazemos a junção dos dados, pelo código de municípios. 

https://www.ibge.gov.br/geociencias/organizacao-do-territorio/estrutura-territorial/23701-divisao-territorial-brasileira.html?=&t=acesso-ao-produto

In [ ]:
oobr.load_geoses(uf=["nacional"])

In [ ]:
aux  = oobr.geoses_planilhas['nacional'].merge(oobr.df_cod_mun_ibge, left_on='v0002_codigo_do_municipio', right_on='Código Município Completo')
list(aux.columns)

In [ ]:
#!cp '/content/drive/MyDrive/workdir/OOBr/reprocessamento/Dados Obstetricos/Dados Wesley - municípios/dados_indicadores-completo_muni/Indicadores_Obstetricos_muni.xlsx' .
df_indicadores_obstetricos_muni = pd.read_excel("/content/drive/MyDrive/workdir/OOBr/reprocessamento/Dados Obstetricos/Dados Wesley - municípios/dados_indicadores-completo_muni/Indicadores_Obstetricos_muni.xlsx").query('Ano==2010')

In [ ]:
aux2 = df_indicadores_obstetricos_muni.merge(aux, left_on='Codigo', right_on='cod_6')
aux2.dtypes

In [ ]:
cols = [ 'Nascimentos', 
         'Porc_prematuros','Porc_cesareas', 'Porc_grav_multipla', 'Porc_anomalias',
         'Porc_nenhuma_consulta', 'Porc_consulta7mais', 'Porc_feminino',
         'Porc_raca_mae_branca', 'Porc_raca_mae_negra', 'Porc_peso_menor_2500',
         'Porc_apgar1_menor_7', 'Porc_apgar5_menor_7', 'Taxa_mort_materna',
         'Taxa_mort_infantil', 'Taxa_mort_neonatal',
         'Taxa_mort_neonatal_precoce', 'Taxa_mort_neonatal_tardia',
         'Taxa_mort_pos_neonatal', 'v0002_codigo_do_municipio', 'GeoSES',
          'educação','mobilidade', 'pobreza', 'privacao_material', 'riqueza', 'renda',
       'segregacao',  
       'Região Geográfica Intermediária',
       'Região Geográfica Imediata',
       'Mesorregião Geográfica',
       'Microrregião Geográfica', 
       'Município', 'Código Município Completo',  
        'Codigo']

newCols=[]
for c in cols:
  newCols.append(c+"_")
  aux2[newCols[-1]] = (aux2[c] - aux2[c].mean())/ aux2[c].std()
clusteriza_benchmark(aux2[newCols].fillna(value=0), "GeoSES - Nacional + Indicadores Obstétricos")

In [ ]:
kmeans = KMeans(n_clusters=5, random_state=0).fit(aux2[newCols].fillna(value=-2))
aux2['Clusters'] = kmeans.labels_

In [ ]:
aux2.to_csv("/content/drive/MyDrive/workdir/OOBr/reprocessamento/Dados Obstetricos/Dados Wesley - municípios/dados_indicadores-completo_muni/merge_geoses_indicadores_obstetricos.csv", sep=';', index=False)

In [ ]:
divisoes_geograficas = ['Nome Região Geográfica Imediata','Nome Região Geográfica Intermediária','Nome_Mesorregião','Nome_Microrregião']
for d in divisoes_geograficas:
  aux.groupby(d)[['GeoSES']].mean().sort_values(by="GeoSES").plot(kind='barh', figsize=(18.16,10))


In [ ]:
oobr.geoses_planilhas.keys()

In [ ]:
def insere_nome(df):
  aux  = []
  for id in df['v0002_codigo_do_municipio'].unique():
    aux.append(oobr.get_nome_municipio(id))
  df['mun'] = aux


for uf in tqdm(oobr.geoses_planilhas.keys()):
  insere_nome(oobr.geoses_planilhas[uf])


In [ ]:
for uf in tqdm(oobr.geoses_planilhas.keys()):
  oobr.geoses_planilhas[uf].to_csv(uf+"_mun.csv", sep=';', index=False)

In [ ]:
!mkdir geoses_nomes_mun
!mv *_mun.csv geoses_nomes_mun

In [ ]:
!tar -cvf geoses_nomes_mun.tar.gz ./geoses_nomes_mun/

## Load GeoSES com Nome de Municípios

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!cp /content/drive/MyDrive/workdir/OOBr/reprocessamento/geoses_nomes_mun.tar.gz .
!cp /content/drive/MyDrive/workdir/OOBr/reprocessamento/porcentagem_morte_materna_gravidez_adolescencia.csv .
!cp /content/drive/MyDrive/workdir/OOBr/reprocessamento/df_consolida_1605_2.csv .
!cp /content/drive/MyDrive/workdir/OOBr/reprocessamento/dados_brutos_com_latitudes_obstetricos_ibge.csv .


df_obstibge = pd.read_csv("dados_brutos_com_latitudes_obstetricos_ibge.csv", sep=';').query("Ano==2010")[['Nascimentos', 'Porc_prematuros', 'Porc_cesareas',
                                                                                            'Porc_grav_multipla', 'Porc_anomalias', 'Porc_nenhuma_consulta',
                                                                                            'Porc_consulta7mais', 'Porc_feminino', 'Porc_raca_mae_branca',
                                                                                            'Porc_raca_mae_negra', 'Porc_peso_menor_2500', 'Porc_apgar1_menor_7',
                                                                                            'Porc_apgar5_menor_7', 'Ano', 'UF',
                                                                                            'Estado']]
df_consolida = pd.read_csv("df_consolida_1605_2.csv", sep=';')
df_morte_grav = pd.read_csv("porcentagem_morte_materna_gravidez_adolescencia.csv", sep=';')
df_porc_gravidez_adolescencia = pd.read_csv("porcentagem_morte_materna_gravidez_adolescencia.csv", sep=';')
df_morte_gravidez_adolescencia = pd.read_csv("df_morte_menor_10_maiorIgual_idade_menorIgual_20.csv", sep=';')

In [ ]:
df_obstibge.head()

In [ ]:
!tar -xf ./geoses_nomes_mun.tar.gz

In [ ]:
!head -n 1 ./geoses_nomes_mun/AC_mun.csv

In [ ]:
pasta="./geoses_nomes_mun/"
all_files = os.listdir(pasta)    
planilhas_sim = {}
csv_files = list(filter(lambda f: f.endswith('.csv'), all_files))
csv_files.sort()
csv_files[:3]

In [ ]:
df_obstibge.set_index('UF').loc['AP']

In [ ]:
def melhores_valores_k_geoses():
  melhores_valores_k_geoses = []
  dic_dfs = {}
  for uf in csv_files:
    uf_ = uf.replace("_mun.csv", "")
    aux = pd.read_csv(pasta+uf, sep=';').set_index('mun')[[ 'GeoSES',
                                                            'educação',
                                                            'mobilidade',                                   
                                                            'pobreza',
                                                            'privacao_material',
                                                            'riqueza', 
                                                            'renda', 
                                                            'segregacao',
                             ]]
  
  
    if uf_ !="DF":
      #print( "Estado: %s Municípios: %d"   %(uf_, aux.shape[0]))
      #clusteriza_benchmark(aux, uf_)
      #melhores_valores_k_geoses.append([uf_, aux.shape[0]]+
      _ = melhor_valor_k_(aux, uf_)[0][:3]
      _ = [uf_, aux.shape[0]] + _
      clusteriza_benchmark(aux, uf_)
      
    
    
      #print(uf_)
      #print(aux.shape[0], aux.dropna().shape[0], _[2])
    #if uf_ !="DF":

      kmeanModel = KMeans(n_clusters=_[2]).fit(aux.dropna())
      aux['Cluster'] = kmeanModel.labels_
      dic_dfs[uf_] = aux.copy()
      list_cluster = aux['Cluster'].value_counts().sort_index().values.tolist()
      if len(list_cluster)==4:
        list_cluster.append(0)
      _ = _ + list_cluster
      
      melhores_valores_k_geoses.append(_)
      print(_)





    df_melhores_valores_k_geoses = pd.DataFrame(melhores_valores_k_geoses, columns=['uf', 'municípios',
                                                                                    'melhor k', 'mu_distancias',
                                                                                    'std_distancias',
                                                                                    'Cluster 0', 'Cluster 1',
                                                                                    'Cluster 2', 'Cluster 3',
                                                                                    'Cluster 4' 
                                                                                    ]).sort_values(by=['municípios'], ascending=True).set_index("uf")#.reset_index().drop('index', axis=1)
    
  
  return round(df_melhores_valores_k_geoses,2),dic_dfs 


retorno = melhores_valores_k_geoses()

In [ ]:
retorno[0]

In [ ]:
import seaborn as sea
fator = .5
fig_dims = (18.16*fator,10*fator)
fig, ax = plt.subplots(figsize=fig_dims)  

sea.lineplot(data=retorno[0][['municípios',  'Cluster 0', 'Cluster 1', 'Cluster 2', 'Cluster 3', 'Cluster 4']],ax=ax)
sea.scatterplot(data=retorno[0][['municípios',  'Cluster 0', 'Cluster 1', 'Cluster 2', 'Cluster 3', 'Cluster 4']],ax=ax, legend=False)
plt.grid()
plt.tight_layout()
plt.savefig("./imgOOBr/cluster_por_estado.png")

## GeoSES - Análise de Agrupamento

In [ ]:
def print_heatmap(df, titulo):
  _ = df
  fator = .75
  fig_dims = (18.16*fator,10*fator)
  fig, ax = plt.subplots(figsize=fig_dims)  
  sea.heatmap(data=_[_>=.5],annot=True, cmap='Blues' ,  ax=ax)
  sea.heatmap(data=_[_<=-.5],annot=True, cmap='Reds' ,  ax=ax)
  plt.title(titulo)
  plt.savefig("./imgOOBr/heatmap"+titulo+"_.png")
  plt.show()

def print_boxplot(df, titulo):
  _ = df
  fator = .75
  fig_dims = (18.16*fator,10*fator)
  fig, ax = plt.subplots(figsize=fig_dims)  
  sea.boxplot(data=_,  ax=ax)  
  plt.title(titulo)
  plt.savefig("./imgOOBr/boxplot_"+titulo+"_.png")
  plt.show()


for uf in retorno[1].keys():
  print("####################################################################")
  feats = [   'GeoSES',
              'educação',
              'mobilidade',                                   
              'pobreza',
              'privacao_material',
              'riqueza', 
              'renda', 
              'segregacao',
            ]
  df = retorno[1][uf]
  

  _ = df[feats].corr(method='spearman')
  _ = _[_<1]
  _ = _[_>-1]
  tit = "".join(["GeoSES - Panorama Estadual - ", uf])
  print_heatmap(_,tit )
  print_boxplot(df, tit)
  for clu in df['Cluster'].unique():
    df_ = df.query('Cluster=='+str(clu))[feats].copy()
    _ = df_.corr(method='spearman')
    _ = _[_<1]
    _ = _[_>-1]
    print(uf, clu, _.shape)
    tit = "".join(["GeoSES - ", uf, " - Perspectiva por Cluster "+str(clu)])
    print_heatmap(_,tit )
    print_boxplot(df_, tit)


In [ ]:
!mv imgOOBr  /content/drive/MyDrive/workdir/OOBr/reprocessamento/

## Seleção das Variáveis Componentes do Indicador

In [ ]:
import seaborn as sea

fator = .5
fig_dims = (10*fator,18.16*fator)
fig, ax = plt.subplots(figsize=fig_dims)  

_ = oobr.geoses_planilhas['ES'].set_index('mun')[['GeoSES',
                                      'educação',
                                      'mobilidade',                                   
                                      'pobreza',
                                      'privacao_material',
                                      'riqueza', 
                                      'renda', 
                                      'segregacao',
                             ]]#.T.corr()
sea.barplot(data=_, orient='v')

#sea.heatmap(data=_[_>=.5],annot=True, cmap='Greens' ,  ax=ax)
#sea.heatmap(data=_[_<=-.5],annot=True, cmap='Reds' ,  ax=ax)

# Base SIM - Carga das Planilhas - Com base na UF e/ou Ano

In [ ]:
oobr.sim_load(uf=['ES'], ano = [2009],  proporcao=100)

In [ ]:
print(oobr.sim_planilhas.keys())

In [ ]:
oobr.sim_planilhas['ES_2009'].head()

In [ ]:
oobr.sim_planilhas['ES_2009'].shape

# Listagem das planilhas carregadas em memória

In [ ]:
!wc -l ./workdir/ETLSIM.DORES_ES_2009_t.csv

In [ ]:
list(oobr.sim_planilhas.keys())

## Dicionário de Dados

In [ ]:
oobr.dict_SIM.sort_values(by='name')

## Filtros para a base SIM

OBITOGRAV e OBITOPUERP iguais a 1 (variáveis que indicam óbito na gravidez/parto/puerpério) pras categorias B20-B24, D39.2, E23.0 e M83.0

pras outras, você tem que pegar todos os casos, independentemente de estar marcado como óbito na gravidez/parto/puerpério ou não 

você vai perceber que em alguns anos (acho que em 9), os dados dão um ou dois óbitos a mais

isso é por conta dessa categoria F53, se você exige OBITOGRAV e OBITOPUERP iguais a 1 pra ela os números vão ser menores, mas se você não exigir eles dão a mais pra alguns anos 



In [ ]:
import matplotlib.pyplot as plt

oobr.sim_planilhas['ES_2009']['CAUSABAS_O'].value_counts()

ft_filter = ['CAUSABAS', 'CAUSABAS_O', 'causabas_capitulo', 'causabas_categoria', 'causabas_subcategoria']

for i in ft_filter:
  _ = oobr.sim_planilhas['ES_2009'][i].value_counts()
  print(i)
  print(_.shape)
  #_.tail(50).plot(kind='barh', title=i, figsize=(18.16,10))  
  #plt.show()

In [ ]:
cids_causa_materna =  list(map(lambda x: "".join(['O0',str(x)]) , range(0,10))) + \
                      list(map(lambda x: "".join(['O',str(x)]) , range(10,96))) + \
                      list(map(lambda x: "".join(['O',str(x)]) , range(98,100))) +\
                      list(map(lambda x: "".join(['B',str(x)]) , range(20,25))) + ['A34','D392', 'E230', 'F53', 'M83']

In [ ]:
for ft in ft_filter:
  print(ft)
  planilha = oobr.sim_planilhas['ES_2009']
  pattern = "|".join(cids_causa_materna)
  mask = planilha[ft].str.contains(pattern, case=False, na=False)
  print(planilha[mask].shape)
#planilha = oobr.sim_planilhas[edicao]['CAUSABAS'].isin([cids])
#planilha = oobr.sim_planilhas[edicao].query("OBITOPARTO ==1 or OBITOPUERP==1 or  OBITOGRAV==1")
#print(planilha.shape)
#print("Metadados da Planilha %s" % edicao)
#_ = list(planilha.columns)
#_.sort()
#mask = planilha['causabas_categoria'].str.contains(pattern, case=False, na=False)
#planilha[mask].shape




In [ ]:
import sys

def scan_sim_contabiliza_obitos():  
  cids_causa_materna =  list(map(lambda x: "".join(['O0',str(x)]) , range(0,10))) + \
                      list(map(lambda x: "".join(['O',str(x)]) , range(10,96))) + \
                      list(map(lambda x: "".join(['O',str(x)]) , range(98,100))) +\
                      list(map(lambda x: "".join(['B',str(x)]) , range(20,25))) + ['A34','D392', 'E230', 'F53', 'M83']
  #for c in cids_causa_materna:
  #  print(c)

  estados = []
  anos = []
  for l in oobr.lista_planilhas:
    ls = l.split("_")
    if len(ls)>2:    
      if ls[1] not in estados:
        estados.append(ls[1])
      if ls[2] not in anos:
        anos.append(ls[2])
  print(estados)
  print(anos)
  
  
  
  morte_menor_20  = {}

  for uf in estados:
    #for ano in anos:
    for ano in ['2010',]:
      
      edicao= "_".join([uf,ano]) #'SP_2017'#    list(oobr.sim_planilhas.keys())[0]
      oobr.sim_load(uf=[uf], ano=[ano], proporcao=100)
      #print(oobr.sim_planilhas[edicao].shape)
      planilha = oobr.sim_planilhas[edicao]#.query("OBITOPUERP ==1 or OBITOGRAV==1")
      
      try:
        ft = "CAUSABAS"
        pattern = "|".join(cids_causa_materna)
        mask = planilha[ft].str.contains(pattern, case=False, na=False)

        planilha = planilha[mask].copy()
        print()
        planilha_ = planilha[mask].query("idade_obito>=10 and idade_obito<=20").copy()
        morte_menor_20[edicao]=planilha_['def_raca_cor'].value_counts() / planilha.shape[0]
      except:
        pass
        #morte_menor_20[edicao]= 0 
        
      #planilha_ = oobr.sim_planilhas[edicao].query("OBITOPUERP==1 or  OBITOGRAV==1")
      #planilha = oobr.sim_planilhas[edicao][oobr.sim_planilhas[edicao]['CAUSABAS'].isin([cids])]
      #planilha = planilha_.query("idade_obito<=15").copy()
      #planilha['def_raca_cor'].value_counts()
      #morte_menor_20[edicao]=planilha['def_raca_cor'].value_counts() / planilha.shape[0] 
      #morte_menor_20[edicao]=planilha['def_raca_cor'].value_counts() / planilha_.shape[0]
  return morte_menor_20     



morte_menor_20 = scan_sim_contabiliza_obitos()





In [ ]:
#df_morte_menor_20 = pd.DataFrame(morte_menor_20).T.reset_index()
df_morte_menor_15 = pd.DataFrame(morte_menor_20).T.reset_index()
ano = []
estado = [] 
for i in df_morte_menor_15['index'].values:
  _ = i.split("_")
  ano.append(_[1])
  estado.append(_[0])

df_morte_menor_15['UF'] = estado
df_morte_menor_15['Ano'] = ano
df_morte_menor_15['Não Branca'] = df_morte_menor_15[['Amarela', 'Indígena', 'Parda', 'Preta']].sum(axis=1)
#df_morte_menor_15.to_csv("df_morte_menor_15.csv", sep=';', index=False)

In [ ]:
estados = []
anos = []
for l in oobr.lista_planilhas:
  ls = l.split("_")
  if len(ls)>2:    
    if ls[1] not in estados:
      estados.append(ls[1])
    if ls[2] not in anos:
      anos.append(ls[2])
anos

In [ ]:

def scan_sinasc():
  estados = []
  anos = []
  for l in oobr.lista_planilhas:
    ls = l.split("_")
    if len(ls)>2:    
      if ls[1] not in estados:
        estados.append(ls[1])
      if ls[2] not in anos:
        anos.append(ls[2])
  morte_menor_20  = {}
  valores = []
  print(anos, estados)
  for uf in tqdm(estados):
    #for ano in anos:        
    for ano in ['2010',]:        
      print("...............")
      print("Ano UF", ano, uf)
    
      edicao= "_".join([uf,ano]) #'SP_2017'#    list(oobr.sim_planilhas.keys())[0]
      oobr.sinasc_load(uf=[uf], ano=[ano], proporcao=100)
      #print(oobr.sinasc_planilhas[edicao].shape)
      planilha = oobr.sinasc_planilhas[edicao].copy() #.query("OBITOPUERP ==1 or OBITOGRAV==1")
      
      print("Tamanho Planilha", planilha.shape)
      print("...............")

      

        #ft = "CAUSABAS_O"
        #pattern = "|".join(cids_causa_materna)

        #mask = planilha[ft].str.contains(pattern, case=False, na=False)

        #planilha = planilha[mask].copy()
        #planilha_ = planilha[mask].query("idade_obito<=15").copy()
        #morte_menor_20[edicao]=planilha_['def_raca_cor'].value_counts() / planilha.shape[0]
      try:
        porc_gravidez_adolescencia = planilha.query("IDADEMAE >=10 and IDADEMAE<=20").shape[0]/\
        planilha.shape[0]
      except:
        porc_gravidez_adolescencia = np.NaN
      try:
        porc_gravidez_multipla = planilha.query("GRAVIDEZ >=2").shape[0]/\
        planilha.shape[0]
      except:
        porc_gravidez_multipla = np.NaN
      try:
        porc_mae_raca_branca = planilha.query("RACACOR ==1").shape[0]/\
        planilha.shape[0]
      except:
        porc_mae_raca_branca = np.NaN
      try:
        porc_mae_raca_nao_branca = planilha.query("RACACOR !=1").shape[0]/\
        planilha.shape[0]
      except:
        porc_mae_raca_nao_branca = np.NaN
      try:
        porc_peso_menor_2500 = planilha.query("PESO <2500").shape[0]/\
        planilha.shape[0]
      except:
        porc_peso_menor_2500 = np.NaN
      try:
        porc_consulta_7_mais = planilha.query("CONSULTAS == 4").shape[0]/ planilha.shape[0]
      except:
        porc_consulta_7_mais = np.NaN      
      try:        
        porc_nenhuma_consulta =  planilha.query("CONSULTAS==1 ").shape[0]/\
        planilha.shape[0]
      except:
        porc_nenhuma_consulta = np.NaN
      try:
        porc_apgar1_menor_7 = planilha.query("APGAR1 <7").shape[0]/\
        planilha.shape[0]
      except:
        porc_apgar1_menor_7 = np.NaN      
      try:
        porc_apgar5_menor_7 = planilha.query("APGAR5 <7").shape[0]/\
        planilha.shape[0]
      except:
        porc_apgar5_menor_7 = np.NaN
      try:
        porc_cesareas = planilha.query("PARTO ==2").shape[0]/\
        planilha.shape[0]
      except:
        porc_cesareas = np.NaN
      try:
        porc_anomalias = planilha.query("IDANOMAL ==1").shape[0]/\
        planilha.shape[0]
      except:
        porc_anomalias = np.NaN
      try:
        porc_prematuros = planilha.query("parto_prematuro >0").shape[0]/\
        planilha.shape[0]
      except:
        porc_prematuros = np.NaN      
      try:
        porc_feminino = planilha.query("SEXO ==2").shape[0]/\
        planilha.shape[0]
      except:
        porc_feminino = np.NaN
      try:
        quantidade_nascimentos = planilha.shape[0]
      except:
        quantidade_nascimentos = np.NaN
        
      val = [uf,ano]
      for v in [
                porc_gravidez_adolescencia,
                porc_gravidez_multipla,
                porc_mae_raca_branca, 
                porc_mae_raca_nao_branca,
                porc_peso_menor_2500,
                porc_consulta_7_mais, 
                porc_nenhuma_consulta,
                porc_apgar1_menor_7,
                porc_apgar5_menor_7, 
                porc_cesareas, 
                porc_anomalias,
                porc_prematuros,
                porc_feminino,
                quantidade_nascimentos
                ]:
                v = round(v, 4)
                val.append(v)
              

      print(val)
      valores.append(val)
        #print(valores[-1])
  return      valores


morte_menor_20 = scan_sinasc()  



df_consolida = pd.DataFrame(morte_menor_20, columns=['uf',
                                                      'ano',
                                                      'porc_gravidez_adolescencia',
                                                      'porc_gravidez_multipla',
                                                      'porc_mae_raca_branca', 
                                                      'porc_mae_raca_nao_branca',
                                                      'porc_peso_menor_2500',
                                                      'porc_consulta_7_mais', 
                                                      'porc_nenhuma_consulta',
                                                      'porc_apgar1_menor_7',
                                                      'porc_apgar5_menor_7', 
                                                      'porc_cesareas', 
                                                      'porc_anomalias',
                                                      'porc_prematuros',
                                                      'porc_feminino',
                                                      'quantidade_nascimentos'   
                                                     ])
_ = df_consolida.pivot_table(index=['uf'], columns=['ano'],  values=['porc_gravidez_adolescencia'])
_

In [ ]:
df_consolida.shape[0]/27

In [ ]:
df_consolida[['uf', 'ano', 'porc_gravidez_adolescencia']].to_csv("porc_gravidez_adolescencia.csv", sep=';', index=False)

In [ ]:
df_morte_menor_15[['Branca','Não Branca', 'UF', 'Ano' ]]

In [ ]:
df_consolida.to_csv("df_consolida_1605_2.csv", sep=';', index=False)


In [ ]:
regioes = {'Região Norte': [{"nome": "Acre", "sigla": "AC"},
                     {"nome": "Amapá", "sigla": "AP"},
                     {"nome": "Amazonas", "sigla": "AM"},
                     {"nome": "Rondônia", "sigla": "RO"},
                     {"nome": "Roraima", "sigla": "RR"},
                         {"nome": "Pará", "sigla": "PA"},
                     {"nome": "Tocantins", "sigla": "TO"}
        
                      ],
           'Região Centro-Oeste':[   {"nome": "Distrito Federal", "sigla": "DF"},
                              {"nome": "Goiás", "sigla": "GO"},
                              
                              {"nome": "Mato Grosso", "sigla": "MT"},
                              {"nome": "Mato Grosso do Sul", "sigla": "MS"}],
           'Região Nordeste': [{"nome": "Maranhão", "sigla": "MA"},
                        {"nome": "Alagoas", "sigla": "AL"},
                        {"nome": "Bahia", "sigla": "BA"},
                        {"nome": "Ceará", "sigla": "CE"},
                        {"nome": "Paraíba", "sigla": "PB"},
                        {"nome": "Rio Grande do Norte", "sigla": "RN"},
                        
                        {"nome": "Pernambuco", "sigla": "PE"},
                        {"nome": "Piauí", "sigla": "PI"},
                        {"nome": "Sergipe", "sigla": "SE"},
                      ],
           'Região Sudeste': [ {"nome": "Rio de Janeiro", "sigla": "RJ"},
                        {"nome": "Espírito Santo", "sigla": "ES"},
                        {"nome": "São Paulo", "sigla": "SP"}, 
                        {"nome": "Minas Gerais", "sigla": "MG"}],
           'Região Sul': [ 
                    {"nome": "Rio Grande do Sul", "sigla": "RS"},
                   {"nome": "Paraná", "sigla": "PR"},
                    {"nome": "Santa Catarina", "sigla": "SC"}]
                    }                   

In [ ]:
import seaborn as sea
import matplotlib.pyplot as plt

def plot_uf_serie_raca(ufs, regiao, branca=1):
  if branca ==1:
    ft = "Branca"
  elif branca == -1:
    ft = "Ignorado"
  else:
    ft = 'Não Branca'

  fator = .75
  dimA = 18.16 * fator
  dimB = 10  * fator
  fig, ax = plt.subplots(figsize=(dimA, dimB))
  for uf in ufs:  
    sea.lineplot(data=df_morte_menor_15.query("UF=='"+uf+"'"), x='Ano', y=ft, label=uf, ax=ax,palette='Dark2')
    sea.scatterplot(data=df_morte_menor_15.query("UF=='"+uf+"'"), x='Ano', y=ft,  ax=ax, palette='Dark2')
    
  if regiao == 'Brasil':
    plt.title(regiao + ' - Percentual de Mortes Maternas - Raça ' + ft + ' - Idade Maior que 10 e  Menor que 21 Anos. Fonte: PCDaS/SIM')
  else:
    plt.title(regiao + ' - Percentual de Mortes Maternas - Raça ' + ft + ' - Idade Maior que 10 e  Menor que 21 Anos. Fonte: PCDaS/SIM')
  plt.grid()
  plt.tight_layout()
  plt.show()



def plot_porc_morte_materna_menos20():

  for r in regioes.items():
    print(r[0])
    _ = []
    for uni_federativa in r[1]:
      _.append(uni_federativa['sigla'])
    
    
    plot_uf_serie_raca(_, r[0])
    #plt.title(r[0])
    #plt.show()

def plot_porc_morte_materna_menos20_():

  for r in regioes.items():
    print(r[0])
    _ = []
    for uni_federativa in r[1]:
      _.append(uni_federativa['sigla'])
    
    
    plot_uf_serie_raca(_, r[0],0 )
    #plt.title(r[0])
    #plt.show()


def plot_porc_morte_materna_menos20__():

  for r in regioes.items():
    print(r[0])
    _ = []
    for uni_federativa in r[1]:
      _.append(uni_federativa['sigla'])
    
    
    plot_uf_serie_raca(_, r[0],-1 )
    #plt.title(r[0])
    #plt.show()




plot_uf_serie_raca(estados,'Brasil')
plot_uf_serie_raca(estados,'Brasil', 0)
plot_uf_serie_raca(estados,'Brasil', -1)
  

In [ ]:
plot_porc_morte_materna_menos20()

In [ ]:
plot_porc_morte_materna_menos20_()

In [ ]:
plot_porc_morte_materna_menos20__()

In [ ]:

fator = 2
fig, ax = plt.subplots(figsize=(18.16*fator,10*fator ))

#sea.heatmap(data= df_morte_menor_20.pivot_table(columns=['UF'],index=['Ano'],  values=['Amarela','Ignorado', 'Indígena', 'Parda','Branca',  'Preta']), ax=ax, cmap='Purples', annot=True)
_ = df_morte_menor_15.pivot_table(index=['UF'], columns=['Ano'], values=['Branca', 'Não Branca','Ignorado'])
_ = _[_>0]

sea.heatmap(data= round(_,2) , ax=ax, cmap='Pastel1', annot=True,linewidths=.3, annot_kws={"size": 8})
plt.title('Percentual de Mortes Maternas - Idade Menor que 21 Anos. Fonte: PCDaS/SIM')
plt.show()




In [ ]:
df_morte_menor_15

In [ ]:
df_morte_menor_15[df_morte_menor_15['UF']==uf]['Branca'].dropna()

In [ ]:
df_morte_menor_15.to_csv("porcentagem_morte_materna_gravidez_adolescencia.csv", sep=';', index=False)

## Teste de Postos - Mann Kendall 


https://github.com/mmhs013/pyMannKendall/blob/master/README.md

In [ ]:
#perform Mann-Kendall Trend Test
!pip install pymannkendall
import pymannkendall as mk
from statsmodels.graphics.gofplots import qqplot
from scipy.stats import shapiro
import seaborn as sea


In [ ]:
from logging import exception
kendall = []

for ft in ['Branca', 'Não Branca','Ignorado']:

  for uf in df_morte_menor_15['UF'].unique():  
    x = df_morte_menor_15[df_morte_menor_15['UF']==uf][ft].dropna()
    try:

      trend,h,p,z,tau,s,var_s,slope,intercept = mk.original_test(x)
      #trend,h,p,z,tau,s,var_s,slope,intercept = mk.(x)
      stat, p_ = shapiro(x)
      alpha = 0.05
      kendall.append([uf,ft,  trend,h,p,z,tau,s,var_s,slope,intercept, stat, p_, p > alpha])

      

      #qqplot(x, line='s')
    
      #if p > alpha:
      #  print('Sample looks Gaussian (fail to reject H0)')
      #else:
      #  print('Sample does not look Gaussian (reject H0)')
      #plt.show()
    except:
      pass


In [ ]:
df_kendall = pd.DataFrame(kendall, columns=['estado','raça',  'trend','h','p','z','tau','s','var_s','slope','intercept', 'stat', 'p_', 'h0'])

df_kendall = round(df_kendall.sort_values(by=['trend','p', 'intercept'], ascending=True),3)
df_kendall.to_csv("teste_significancia_porcentagem_morte_materna_gravidez_adolescencia.csv", sep=';', index=False)



In [ ]:
for grup in ['trend']:
  estados_sample = list(df_kendall.groupby(grup).sample(1)['estado'].values)
  estados_sample.sort()

  #plot_uf_serie(estados_sample, "Amostras por Comportamento ")

  for ft, param  in zip(['Branca', 'Não Branca','Ignorado'], [1, 0, -1]):
    #plot_uf_serie_raca(estados_sample, "Amostras de Tendência [por "+ grup+"]", param)
    plot_uf_serie_raca(estados_sample, "Amostras de Tendência ", param)



    df_morte_menor_15['Ano']=df_morte_menor_15['Ano'].astype(int)
    _ = df_morte_menor_15[['Ano', ft, 'UF']].dropna().copy()

    _ = _[_['UF'].isin(estados_sample)]

    sea.lmplot(data=_, x='Ano', y=ft, hue='UF', palette='Dark2', aspect=2.5)
    plt.title("Regressão Linear - Percentual de Mortalidade")
    plt.grid()
    plt.tight_layout()
    plt.show()

  


In [ ]:
estados_sample

In [ ]:
df_kendall

In [ ]:
#fig, ax = plt.subplots(figsize=(30, 40))

!pip install --upgrade youtube-dl

from __future__ import unicode_literals
import youtube_dl

ydl_opts = {}
with youtube_dl.YoutubeDL(ydl_opts) as ydl:
    ydl.download(['https://www.youtube.com/watch?v=o6Rmc-bcUiU'])

In [ ]:
df_morte_menor_20

In [ ]:
oobr.lista_planilhas

## Funções de Clusterização e Calculo do Melhor valor de K

In [ ]:
from sklearn.cluster import KMeans
from sklearn import metrics
from scipy.spatial.distance import cdist
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np


def clusteriza_benchmark(X, titulo):
  rodada_inertia = {}
  rodada_distocao = {}
  metricas = ['euclidean', 
              #'minkowski',
              'cityblock',
             # 'seuclidean',
             # 'sqeuclidean',
              'cosine', 
              'correlation',
              #'hamming',
              #'jaccard',
              #'jensenshannon',
              #'chebyshev',
              #'canberra',
              #'braycurtis',
              'mahalanobis',
              #'yule',
              #'matching',
              #'dice',
              #'kulsinski',
              #'rogerstanimoto', 
              #'russellrao',
              #'sokalmichener', 
              #'sokalsneath',
              #'wminkowski',
              ]
  
  for met in tqdm(metricas):
    distortions = []
    inertias = []
    mapping1 = {}
    mapping2 = {}
    K = range(2, 9)

    for k in tqdm(K):
        #print("%s %d" %(met, k))
        # Building and fitting the model
        kmeanModel = KMeans(n_clusters=k).fit(X)
        kmeanModel.fit(X)
        distortions.append(sum(np.min(cdist(X, kmeanModel.cluster_centers_,
                                            met), axis=1)) / X.shape[0])
        inertias.append(kmeanModel.inertia_)
        mapping1[k] = sum(np.min(cdist(X, kmeanModel.cluster_centers_,
                                      met), axis=1)) / X.shape[0]
        mapping2[k] = kmeanModel.inertia_
    mpl1 = []
    for key, val in mapping1.items():
      
      mpl1.append(val)    
    mp1 = pd.Series(mpl1.copy())  
    rodada_distocao[met] = (mp1 - mp1.mean())/mp1.std()

    mpl2 = []
    for key, val in mapping2.items():    
      mpl2.append(val)

    mp2 = pd.Series(mpl2.copy())  
    rodada_inertia[met] = (mp2 - mp2.mean())/mp2.std()

  dfdistortion = pd.DataFrame.from_dict(rodada_distocao)
  dfdistortion.index=list(K)

  dfinertia = pd.DataFrame.from_dict(rodada_inertia)
  dfinertia.index=list(K)  
  
  ax = dfdistortion.plot(title=titulo,figsize=(9.08,5))
  ax.legend(loc='upper left')
  plt.grid(color='grey', linestyle=':', linewidth=.5)
  ax.set(xlabel='Valores de K', ylabel='Distorção Normalizada')  
  plt.tight_layout()
  #salva_imagem_drive(plt, 'linplot Corr. Distorcao: '+ str(titulo))
  
  plt.savefig("./imgOOBr/"+titulo+"_distortion.png")
  plt.show()


  
  ax = dfinertia.plot(title=titulo,figsize=(9.08,5))
  ax.legend(loc='upper left')
  plt.grid(color='grey', linestyle=':', linewidth=.5)
  plt.tight_layout()
  ax.set(xlabel='Valores de K', ylabel='Inércia Normalizada')
  #salva_imagem_drive(plt, 'linplot Corr. Inércias: ' + titulo)
  plt.savefig("./imgOOBr/"+titulo+"_correlation.png")
  plt.show()

#os.system("rm ./oobr_output_natalidade_ibge/*.png")

In [ ]:
from math import sqrt
from sklearn.cluster import KMeans

def calculate_wcss(data):
  wcss = []
  for n in range(2, 9):
    kmeans = KMeans(n_clusters=n,algorithm='elkan')
    kmeans.fit(X=data)
    wcss.append(kmeans.inertia_)

  return wcss

def optimal_number_of_clusters(wcss):
  rodadas_cluster = []
  x1, y1 = 2, wcss[0]
  x2, y2 = 11, wcss[len(wcss)-1]

  distances = []
  for i in range(len(wcss)):
    #print('track d',i)
    x0 = i+2
    y0 = wcss[i]
    numerator = abs((y2-y1)*x0 - (x2-x1)*y0 + x2*y1 - y2*x1)
    denominator = sqrt((y2 - y1)**2 + (x2 - x1)**2)
    distances.append(numerator/denominator)
  rodadas_cluster.append([distances.index(max(distances)) + 2,pd.Series(distances).mean(),pd.Series(distances).std(), max(distances), min(distances)])
  

  return rodadas_cluster


def melhor_valor_k_(data, tit):
  sum_of_squares = calculate_wcss(data)
  rodadas_cluster = optimal_number_of_clusters(sum_of_squares) 
  rodadas_cluster.append("".join(tit[0:5])+str(len(tit)))
  dfDistanciasQuadraticasVetores = pd.DataFrame(sum_of_squares)

  dfDisQuadVetScaled = (dfDistanciasQuadraticasVetores - dfDistanciasQuadraticasVetores.mean())  / dfDistanciasQuadraticasVetores.std()    #
  dfDistanciasQuadraticasVetores.columns=['']
  return rodadas_cluster 




In [ ]:
feat_clean = ['Nascimentos', 'Porc_prematuros', 'Porc_cesareas', 'Porc_grav_multipla',
       'Porc_grav_multipla', 'Porc_grav_multipla', 'Porc_grav_multipla',
       'Porc_anomalias', 'Porc_nenhuma_consulta', 'Porc_consulta7mais',
       'Porc_consulta7mais', 'Porc_consulta7mais', 'Porc_consulta7mais',
       'Porc_feminino', 'Porc_raca_mae_branca', 'Porc_raca_mae_negra',
       'Porc_peso_menor_2500', 'Porc_apgar1_menor_7', 'Porc_apgar5_menor_7',
       'GINI', 'T_DES', 'T_ANALF15M', 'PAREDE', 'T_AGUA', 'T_BANAGUA',
       'T_BANAGUA', 'T_BANAGUA', 'T_BANAGUA', 'T_BANAGUA', 'T_BANAGUA',
       'T_BANAGUA', 'T_BANAGUA', 'T_BANAGUA', 'AGUA_ESGOTO', 'AGUA_ESGOTO',
       'AGUA_ESGOTO', 'AGUA_ESGOTO', 'T_LIXO']




## Tree View

In [ ]:
from matplotlib import pyplot as plt
from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier 
from sklearn import tree




#X = dados.sort_index()[feat_list[:-1]]

#X = dados.sort_index()[feat_list_obst]
y = df_resid['Cluster']


clf = DecisionTreeClassifier(random_state=1234,
                             min_samples_split = 2,
                             splitter ='best' , 
                             max_depth=10)
model = clf.fit(df_resid[['MSE', 'RMSE', 'T_STATISTICO', 'ρ-VALUE', 'ρ']],y )

import graphviz
# DOT data
dot_data = tree.export_graphviz(clf, out_file=None, 
                                #feature_names=feat_list[:-1],
                                feature_names= ['MSE', 'RMSE', 'T_STATISTICO', 'ρ-VALUE', 'ρ'],  #feat_list_obst,  
                                #class_names=['Condições Melhores', 'Condições Intermediárias', 'Condições Piores'],
                                filled=True)

# Draw graph
graph = graphviz.Source(dot_data, format="png") 
graph







In [ ]:

!pip install dtreeviz --quiet
from dtreeviz.trees import dtreeviz, explain_prediction_path # remember to load the package

viz = dtreeviz(clf,df_resid[['MSE', 'RMSE', 'T_STATISTICO', 'ρ-VALUE', 'ρ']],y,
                target_name="Clusters dos Resíduos dos Indicadores",               
                feature_names= ['MSE', 'RMSE', 'T_STATISTICO', 'ρ-VALUE', 'ρ'],
                show_root_edge_labels = True,
                
                 #feat_list_obst,  
                
                #class_names=['Condições Melhores', 'Condições Intermediárias', 'Condições Piores'],    
                          
               )

viz


In [ ]:
clf = DecisionTreeClassifier(random_state=1234,
                             min_samples_split = 2,
                             splitter ='best' , 
                             max_depth=10)

model = clf.fit(dados2plot[feat_full] ,dados2plot['Cluster'] )

viz = dtreeviz(clf,dados2plot[feat_full],dados2plot['Cluster'],
                target_name="Clusters dos Resíduos dos Indicadores",               
                feature_names= feat_full,
                #show_root_edge_labels = True,
                
                 #feat_list_obst,  
                
                #class_names=['Condições Melhores', 'Condições Intermediárias', 'Condições Piores'],    
                          
               )

viz

https://reposhub.com/python/deep-learning/parrt-dtreeviz.html

# Conferencia - Valores Antigos e Novos do SIM e SINASC

In [ ]:
!mkdir confere
!mv *.csv confere

In [ ]:
nome_arquivo = 'confere_oobr.zip'
with zipfile.ZipFile(nome_arquivo, 'r') as zip_ref:
      zip_ref.extractall(".")

In [ ]:
all_files = os.listdir("./confere")    
planilhas_sim = {}
csv_files = list(filter(lambda f: f.endswith('.csv'), all_files))
csv_files.sort()
for i in enumerate(csv_files):
  print(i)


In [ ]:
df_confOriginal = pd.read_csv('./confere/'+csv_files[0], sep=';').sort_values(by=["UF", "Ano"])
df_confOriginal#.query("UF=='AC'")#[['UF','Ano', 'Nascimentos']]
#csv_files[8]
df_confOriginal['porcent_anom'] = df_confOriginal['Anomalias'] / df_confOriginal['Nascimentos']
df_confOriginal

In [ ]:
df_confReproces = pd.read_csv('./confere/'+csv_files[11], sep=';')#.drop("Unnamed: 0", axis=1)#.sort_values(by=["UF", "Ano"])
#df_confReproces[['UF','Ano', 'quantidade_nascimentos']]
df_confReproces#.sort_values(by=['ano', 'uf'])[['uf', 'ano','porc_anomalias']]


In [ ]:
import seaborn as sea

sea.jointplot(data=df_confOriginal.merge(df_confReproces, left_on=['UF', 'Ano'], right_on=['uf', 'ano'])[['UF', 'Ano', 'uf', 'ano','porcent_anom', 'porc_anomalias']], x='porcent_anom', y= 'porc_anomalias')

In [ ]:
df_confReproces.columns

In [ ]:
df_confOriginal.columns

In [ ]:
df_confOriginal.query("UF=='AC'")

In [ ]:
df_confOriginal.merge(df_confReproces, left_on=['UF', 'Ano'], right_on=['UF', 'Ano'])[['UF', 'Ano', 'Nascimentos','quantidade_nascimentos' ]]

In [ ]:
#df_gb = pd.read_csv("dados_selecionados-SINASC_UF.csv", sep=';')
df_gb = pd.read_excel("dados_selecionados-SINASC_UF.xlsx")
for i in df_gb.columns[3:]:
  df_gb[i] = df_gb[i]/100

df_gb


In [ ]:
df_confReproces.columns

In [ ]:
merge = df_gb.merge(df_confReproces, right_on=['uf', 'ano'], left_on=['UF', 'Ano'])
list(merge.columns)

In [ ]:
import seaborn as sea
import matplotlib.pyplot as plt



x=['porc_peso_menor_2500',
   'porc_gravidez_multipla', 
   'porc_anomalias', 
   'porc_consulta_7_mais', 
   'porc_nenhuma_consulta', 
   'porc_apgar1_menor_7',
   'porc_apgar5_menor_7',
   'porc_feminino' , 
   'porc_mae_raca_branca',
   'porc_cesareas',
   ]

y=['Porc_peso_menor_2500', 
   'Porc_grav_multipla', 
   'Porc_anomalias',
   'Porc_consulta7mais',
   'Porc_nenhuma_consulta',
   'Porc_apgar1_menor_7', 
   'Porc_apgar5_menor_7', 
   'Porc_feminino', 
   'Porc_raca_mae_branca', 
   'Porc_cesareas',
   
   ]

for k,l in zip(x,y):
  sea.relplot(data=merge, x=k, y=l, hue='UF')
  plt.show()

  sea.residplot(data=merge, x=k, y=l)
  plt.show()

In [ ]:
merge[['Porc_grav_multipla', 'porc_gravidez_multipla']]